# 🧠 Cortex-5: Deep Glass Box & Diagnostics

## 🔬 El Microscopio Profundo
El usuario ha planteado una duda crítica: *"¿Por qué el modelo no genera nada coherente aún?"*.
Para responder, hemos implementado una suite de diagnósticos profundos:

1.  **Sanity Check (Prueba de Cordura)**: Antes de intentar aprender Shakespeare, el modelo debe demostrar que puede memorizar una sola frase ("To be or not to be"). Si no puede hacer esto (Loss -> 0), la arquitectura está rota.
2.  **Atlas de Activación**: Visualizaremos paso a paso qué neuronas se encienden para cada letra. Veremos el "pensamiento" en tiempo real.
3.  **Velocidad del Flujo Residual**: Mediremos cuánto contribuye cada capa al resultado final. ¿Están trabajando todas las capas o algunas son "pasajeros"?

---

In [ ]:
# 0. Configuración
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import requests
from IPython.display import clear_output, display

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Cortex-5 Engine: {device.upper()}")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

In [ ]:
# 1. Arquitectura Instrumentada (Glass Box)
# Modificamos el modelo para que guarde sus estados internos

class MambaBlock(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.in_proj = nn.Linear(d_model, d_model * 2)
        self.out_proj = nn.Linear(d_model, d_model)
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
    def forward(self, x):
        B, L, D = x.shape
        x_and_res = self.in_proj(x)
        x_val, res = x_and_res.chunk(2, dim=-1)
        x_val = x_val.transpose(1, 2)
        x_val = self.conv(x_val)
        x_val = x_val.transpose(1, 2)
        x_val = F.silu(x_val)
        return self.out_proj(x_val * F.sigmoid(res))

class CortexOrganism(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embedding = nn.Embedding(256, config['d_model'])
        self.layers = nn.ModuleList()
        
        # Telemetría
        self.activations = {} # Guardará la salida de cada capa
        self.residual_velocities = [] # Guardará cuánto cambia la señal en cada capa

        for i in range(config['n_layers']): 
            if i % 2 == 0: self.layers.append(MambaBlock(config['d_model']))
            else: self.layers.append(nn.TransformerEncoderLayer(
                d_model=config['d_model'], nhead=config['n_heads'], 
                dim_feedforward=4*config['d_model'], batch_first=True, dropout=0.0
            ))
        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], 256)

    def forward(self, idx, targets=None):
        # Reset telemetría
        self.activations = {}
        self.residual_velocities = []
        
        x = self.embedding(idx)
        
        for i, layer in enumerate(self.layers):
            prev_x = x
            x = layer(x)
            
            # Capturar telemetría
            with torch.no_grad():
                # 1. Activaciones (Neuronas)
                self.activations[f"layer_{i}"] = x.detach().cpu()
                # 2. Velocidad Residual (Norma de la actualización)
                # En Transformer/Mamba estándar es x + layer(x), aquí simplificado para demo
                # Asumimos que la capa devuelve la actualización o el nuevo estado
                diff = (x - prev_x).norm(dim=-1).mean().item()
                self.residual_velocities.append(diff)

        x = self.ln_f(x)
        logits = self.head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

## 🧪 Fase 1: Sanity Check (Overfitting)
Vamos a forzar al modelo a memorizar una sola frase. Si el Loss no baja a 0, no tiene sentido seguir.

In [ ]:
def sanity_check():
    print("🏥 Iniciando Sanity Check...")
    text = "To be or not to be, that is the question."
    data = torch.tensor([ord(c) for c in text], dtype=torch.long).unsqueeze(0).to(device)
    x = data[:, :-1]
    y = data[:, 1:]
    
    # Modelo pequeño para prueba rápida
    config = {'n_layers': 2, 'd_model': 128, 'n_heads': 4, 'backbone': 'hybrid'}
    model = CortexOrganism(config).to(device)
    optim = torch.optim.AdamW(model.parameters(), lr=1e-2)
    
    losses = []
    for i in range(100):
        _, loss = model(x, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        losses.append(loss.item())
        
        if i % 10 == 0:
            print(f"   Iter {i}: Loss {loss.item():.6f}")
            
    plt.plot(losses)
    plt.title("Sanity Check Loss (Debe llegar a 0)")
    plt.show()
    
    if losses[-1] < 0.01:
        print("✅ PRUEBA SUPERADA: El modelo es capaz de aprender.")
        return model, text
    else:
        print("❌ FALLO CRÍTICO: El modelo no puede memorizar una frase simple.")
        return None, None

sanity_model, sanity_text = sanity_check()

## 🔬 Fase 2: Visualización de Dinámica Neuronal
Ahora que sabemos que funciona, veamos **cómo** funciona por dentro usando el modelo del Sanity Check.

In [ ]:
def visualize_neuron_dynamics(model, text):
    if model is None: return
    
    # Correr una inferencia para capturar activaciones
    data = torch.tensor([ord(c) for c in text], dtype=torch.long).unsqueeze(0).to(device)
    model(data) # Forward pass llena self.activations
    
    # 1. Atlas de Activación (Capa 0 - Mamba)
    # Mostramos las primeras 50 neuronas para cada caracter
    act = model.activations['layer_0'].squeeze(0).numpy()[:, :50].T
    
    plt.figure(figsize=(12, 6))
    sns.heatmap(act, cmap='magma', cbar=True)
    plt.title("Atlas de Activación Neuronal (Capa 0: Mamba)")
    plt.xlabel("Secuencia de Texto (Caracteres)")
    plt.ylabel("ID de Neurona (0-50)")
    # Poner los caracteres en el eje X
    plt.xticks(ticks=np.arange(len(text))+0.5, labels=list(text), rotation=0)
    plt.show()
    
    # 2. Velocidad del Flujo Residual
    plt.figure(figsize=(8, 4))
    plt.bar(range(len(model.residual_velocities)), model.residual_velocities, color='#4ade80')
    plt.title("Contribución por Capa (Residual Velocity)")
    plt.xlabel("Capa")
    plt.ylabel("Magnitud del Cambio")
    plt.show()

visualize_neuron_dynamics(sanity_model, sanity_text)

## 🏆 Fase 3: El Gran Torneo (Ahora con Confianza)
Ya sabemos que la arquitectura funciona. Ahora sí, lancemos la búsqueda masiva.

In [ ]:
# Descargar Datos Reales (Shakespeare)
def get_real_data():
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    return requests.get(url).text

raw_text = get_real_data()
data_tensor = torch.tensor([ord(c) for c in raw_text], dtype=torch.long)
train_data = data_tensor[:int(0.9*len(data_tensor))]
val_data = data_tensor[int(0.9*len(data_tensor)):]

def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - 64, (32,))
    x = torch.stack([data[i:i+64] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+65] for i in ix]).to(device)
    return x, y

# Torneo Rápido (Demo)
def run_tournament_demo():
    print("🔥 Iniciando Torneo...")
    results = []
    for i in range(5): # Solo 5 para demo rápida
        config = {
            'n_layers': random.choice([2, 4]),
            'd_model': random.choice([128, 256]),
            'n_heads': 4,
            'learning_rate': 1e-3
        }
        model = CortexOrganism(config).to(device)
        optim = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
        
        # Entrenar 20 iteraciones
        losses = []
        for _ in range(20):
            xb, yb = get_batch()
            _, loss = model(xb, yb)
            optim.zero_grad()
            loss.backward()
            optim.step()
            losses.append(loss.item())
            
        print(f"   Modelo {i} {config}: Loss {losses[-1]:.4f}")
        results.append({'id': i, 'loss': losses[-1], **config})
        
    df = pd.DataFrame(results)
    display(df.sort_values('loss'))

run_tournament_demo()